# 🚌 Addis Ababa Mobility Analyzer

Welcome to the **Ethiopian Data Science & AI Community** starter project!

In this notebook, you'll explore the **road network of Addis Ababa** using real OpenStreetMap data. You’ll learn how to:
- Download and visualize the city's transport network
- Analyze connectivity and central routes
- Identify potential congestion points

🎯 Goal: Understand urban mobility to support smarter transportation planning in Ethiopia.

🔧 Tools: OSMnx, NetworkX, Matplotlib

In [ ]:
# Step 1: Install required libraries
!pip install -q osmnx networkx matplotlib

In [ ]:
# Step 2: Import libraries
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

# Set OSMnx settings
ox.config(log_console=True, use_cache=True)

## Step 3: Download Road Network for Addis Ababa

In [ ]:
# Define the place
place_name = "Addis Ababa, Ethiopia"

# Download the road network
G = ox.graph_from_place(place_name, network_type="drive")

print(f"Downloaded road network for: {place_name}")
print(f"Number of nodes: {len(G.nodes)}")
print(f"Number of edges: {len(G.edges)}")

## Step 4: Visualize the Road Network

In [ ]:
# Plot the full network
plt.figure(figsize=(12, 8))
ox.plot_graph(G, node_size=5, node_color="r", edge_color="gray", edge_linewidth=0.5, bgcolor="white")
plt.title("Road Network of Addis Ababa", fontsize=16)
plt.tight_layout()
plt.show()

## Step 5: Analyze Network Properties

In [ ]:
# Basic network statistics
stats = ox.basic_stats(G)
extended_stats = nx.degree_centrality(G)

print("Basic Network Statistics:")
for key, value in stats.items():
    if key in ['n', 'm', 'density', 'avg_node_degree', 'avg_street_length', 'street_length_total']:
        print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

## Step 6: Find Most Central Roads (Using Betweenness Centrality)

In [ ]:
# Compute betweenness centrality (measures how often a road is on shortest path)
bc = nx.betweenness_centrality(G, weight="length")

# Add to graph as node attribute
for node, bc_val in bc.items():
    if node in G.nodes:
        G.nodes[node]["betweenness"] = bc_val

# Normalize for color scaling
max_bc = max(bc.values())
node_colors = [bc.get(node, 0) / max_bc for node in G.nodes]

# Plot with centrality-based coloring
plt.figure(figsize=(12, 8))
nc = ox.plot_graph_route(
    G, [],
    node_color=node_colors,
    node_size=30,
    node_cmap=plt.cm.viridis,
    edge_color="gray",
    edge_linewidth=0.5,
    bgcolor="white"
)
plt.title("Road Centrality in Addis Ababa (Hotspots = Potential Congestion)", fontsize=14)
plt.tight_layout()
plt.show()

## Step 7: Find Shortest Path Between Two Locations

In [ ]:
# Define two points: Addis Ababa University (Main Campus) and Bole Airport
origin_point = (9.0098, 38.7680)  # AAU
destination_point = (8.9761, 38.7976)  # Bole International Airport

# Find nearest nodes
origin_node = ox.distance.nearest_nodes(G, origin_point[1], origin_point[0])
dest_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# Compute shortest path
route = nx.shortest_path(G, origin_node, dest_node, weight="length")

# Plot route
fig, ax = ox.plot_graph_route(
    G, route,
    route_color="r",
    route_linewidth=6,
    node_size=0,
    bgcolor="white",
    figsize=(12, 8)
)
plt.title("Shortest Driving Route: AAU to Bole Airport", fontsize=16)
plt.show()

## Step 8: Extract Road Data for Further Analysis

In [ ]:
# Convert graph to GeoDataFrame (edges)
gdf_edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

print("First 5 road segments:")
print(gdf_edges[["name", "highway", "length"]].head())

# Save sample to CSV (optional)
gdf_edges[["name", "highway", "length", "geometry"]].to_file("addis_roads_sample.geojson", driver="GeoJSON")
print("\n👉 Road data saved to 'addis_roads_sample.geojson'")

## ✅ Next Steps & Challenges

🔹 **Add real traffic data** (e.g., from GPS, Google Maps, or city surveys)
🔹 **Model peak-hour congestion**
🔹 **Suggest public transit routes** based on connectivity
🔹 **Include walking or biking networks**
🔹 **Translate insights into Amharic** for local planners
🔹 **Contribute your analysis** to the community dataset repo!

📌 Join the [Ethiopian Data Science & AI Community](https://www.linkedin.com/groups/11836061/) to share your findings and collaborate with others!

Let’s build smarter cities — one route at a time. 🇪🇹